[![Binder](https://mybinder.org/badge_logo.svg)](https://notebooks.gesis.org/binder/v2/gh/jolin-io/workshop-accelerate-Python-with-Julia/main?filepath=03-example-cython-vs-cpp-vs-julia.ipynb)

<a href="https://www.jolin.io" target="_blank" rel="noreferrer noopener">
<img src="https://www.jolin.io/assets/Jolin/Jolin-Banner-Website-v1.3-darkmode.webp">
</a>

# **Simulation example:** Python vs Cython vs C++ vs Julia

The code is adapted from the [blog post by The Multi-Agent AI Guy](https://medium.com/agents-and-robots/the-bitter-truth-python-3-11-vs-cython-vs-c-performance-for-simulations-babc85cdfef5)

In [2]:
%%html
<iframe width="560" height="315" src="https://www.youtube.com/embed/hbFrPjeBpqg" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" allowfullscreen>
</iframe>

Again let's activate julia

In [ ]:
from juliacall import Main as jl
%load_ext juliacall.ipython
# JuliaCall comes with its own Julia dependency file juliapkg.json
# however for binder it is much simpler to just reuse binder's installation mechanism
%julia Pkg.activate(Base.current_project())
%julia using PythonCall
%julia set_var(k, v) = @eval $(Symbol(k)) = $v

In [2]:
import math
import random

from datetime import datetime
random.seed(datetime.now().timestamp())

WORLD_WIDTH = 2560
WORLD_HEIGHT = 1440
TIMESTEPS = 2000

jl.set_var("WORLD_WIDTH", WORLD_WIDTH)
jl.set_var("WORLD_HEIGHT", WORLD_HEIGHT)
jl.set_var("TIMESTEPS", TIMESTEPS)

2000

# Python implementation

### Python definition of simulated Agent

In [3]:
class PythonAgent():
    def __init__(self, x=None, y=None, world_width=0, world_height=0):
        super().__init__()

        # default values
        self.vmax = 2.0

        # initial position
        self.world_width = world_width
        self.world_height = world_height
        self.x = x if x else random.randint(0, self.world_width)
        self.y = y if y else random.randint(0, self.world_height)

        # initial velocity
        self.dx = 0
        self.dy = 0

        # inital values
        self.is_alive = True
        self.target = None
        self.age = 0
        self.energy = 0

    def update(self, food=()):
        self.age = self.age + 1

        # we can't move
        if self.vmax == 0:
            return

        # target is dead, don't chase it further
        if self.target and not self.target.is_alive:
            self.target = None

        # eat the target if close enough
        if self.target:
            squared_dist = (self.x - self.target.x) ** 2 + (self.y - self.target.y) ** 2
            if squared_dist < 400:
                self.target.is_alive = False
                self.energy = self.energy + 1

        # agent doesn't have a target, find a new one
        if not self.target:
            min_dist = 9999999
            min_agent = None
            for a in food:
                if a is not self and a.is_alive:
                    sq_dist = (self.x - a.x) ** 2 + (self.y - a.y) ** 2
                    if sq_dist < min_dist:
                        min_dist = sq_dist
                        min_agent = a
            if min_dist < 100000:
                self.target = min_agent

        # initalize 'forces' to zero
        fx = 0
        fy = 0

        # move in the direction of the target, if any
        if self.target:
            fx += 0.1*(self.target.x - self.x)
            fy += 0.1*(self.target.y - self.y)

        # update our direction based on the 'force'
        self.dx = self.dx + 0.05 * fx
        self.dy = self.dy + 0.05 * fy

        # slow down agent if it moves faster than it max velocity
        velocity = math.sqrt(self.dx ** 2 + self.dy ** 2)
        if velocity > self.vmax:
            self.dx = (self.dx / velocity) * (self.vmax)
            self.dy = (self.dy / velocity) * (self.vmax)

        # update position based on delta x/y
        self.x = self.x + self.dx
        self.y = self.y + self.dy

        # ensure it stays within the world boundaries
        self.x = max(self.x, 0)
        self.x = min(self.x, self.world_width)
        self.y = max(self.y, 0)
        self.y = min(self.y, self.world_height)

In [3]:
def main(Agent):

    class Predator(Agent):
        def __init__(self, **kwargs):
            super().__init__(**kwargs)
            self.vmax = 2.5

    class Prey(Agent):
        def __init__(self, **kwargs):
            super().__init__(**kwargs)
            self.vmax = 2.0

    class Plant(Agent):
        def __init__(self, **kwargs):
            super().__init__(**kwargs)
            self.vmax = 0.0

            
    # open the ouput file
    # f = open('output.csv', 'w')
    # print(0, ',', 'Title', ',', 'Predator Prey Relationship / Example 02 / Cython', file=f)

    # create initial agents
    kwargs = dict(
        world_width=WORLD_WIDTH,
        world_height=WORLD_HEIGHT,
    )
    preys = [Prey(**kwargs) for i in range(10)]
    predators = [Predator(**kwargs) for i in range(10)]
    plants = [Plant(**kwargs) for i in range(100)]

    timestep = 0
    while timestep < TIMESTEPS:
        # update all agents
        # no need to update the plants; they do not move
        for a in preys:
            a.update(plants)
        
        for a in predators:
            a.update(preys)

        # handle eaten and create new plant
        plants = [p for p in plants if p.is_alive is True]
        plants = plants + [Plant(**kwargs) for i in range(2)]

        # handle eaten and create new preys
        preys = [p for p in preys if p.is_alive is True]

        for p in preys[:]:
            if p.energy > 5:
                p.energy = 0
                preys.append(Prey(x = p.x + random.randint(-20, 20), y = p.y + random.randint(-20, 20), **kwargs))

        # handle old and create new predators
        predators = [p for p in predators if p.age < 2000]

        for p in predators[:]:
            if p.energy > 10:
                p.energy = 0
                predators.append(Predator(x = p.x + random.randint(-20, 20), y = p.y + random.randint(-20, 20), **kwargs))

        # write data to output file
        #[print(timestep, ',', 'Position', ',', 'Predator', ',', a.x, ',', a.y, file=f) for a in predators]
        #[print(timestep, ',', 'Position',  ',', 'Prey', ',', a.x, ',', a.y, file=f) for a in preys]
        #[print(timestep, ',', 'Position',  ',', 'Plant', ',', a.x, ',', a.y, file=f) for a in plants]

        timestep = timestep + 1

    print(len(predators), len(preys), len(plants))

### Cython definition of Agent

Cython has the unique advantage that you can easily reuse classes within Python

The Cython package was already compiled and is available as `agent` module

In [4]:
from agent import Agent as CythonAgent

### Benchmark Python against Cython class

In [6]:
%timeit -r 3 main(PythonAgent)

0 6 3876
3 212 1851
0 0 4047
0 0 4054
The slowest run took 15.49 times longer than the fastest. This could mean that an intermediate result is being cached.
1.41 s ± 1.65 s per loop (mean ± std. dev. of 3 runs, 1 loop each)


In [7]:
%timeit main(CythonAgent)

2 178 2394
1 5 3892
2 152 2512
2 242 1720
15 173 890
1 98 3022
5 308 726
2 136 2545
257 ms ± 22.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# Full language switch

### Cython

The Cython module `agent` also includes a redefinition of the `main` routine, which gives a little extra speedup.

In [8]:
import agent
%timeit agent.main(WORLD_WIDTH=WORLD_WIDTH, WORLD_HEIGHT=WORLD_HEIGHT, TIMESTEPS=TIMESTEPS)

(0, 2, 3975)
(3, 41, 3286)
(1, 97, 2975)
(1, 188, 2091)
(4, 233, 1575)
(0, 0, 4037)
(1, 366, 888)
(0, 0, 3946)
(3, 117, 2425)
(0, 0, 4059)
(0, 2, 3981)
(1, 67, 3349)
(2, 225, 1863)
(0, 0, 4026)
(0, 0, 4033)
(0, 92, 3271)
(1, 429, 332)
(1, 153, 2647)
(0, 1, 4035)
(0, 5, 3999)
(4, 57, 3025)
(2, 262, 1632)
(0, 1, 4019)
(0, 2, 3986)
(2, 78, 2993)
(2, 209, 2046)
(3, 80, 2964)
(0, 0, 4047)
(0, 2, 3824)
(4, 145, 2311)
(4, 290, 1076)
(5, 243, 1211)
(0, 0, 3973)
(2, 215, 1751)
(3, 258, 1351)
(4, 291, 1017)
(4, 144, 2409)
(2, 45, 3413)
(3, 342, 798)
(1, 160, 2574)
(0, 2, 3978)
(1, 232, 2005)
(4, 272, 1270)
(1, 5, 3976)
(1, 121, 2877)
(0, 6, 3847)
(5, 306, 740)
(0, 49, 3444)
(1, 258, 1696)
(3, 285, 1249)
(0, 2, 4043)
(2, 265, 1670)
(0, 207, 2255)
(0, 0, 4018)
(1, 126, 2876)
(2, 193, 2279)
(6, 343, 326)
(5, 274, 1019)
(0, 187, 2359)
(0, 0, 3975)
(3, 355, 529)
(1, 115, 2848)
(9, 312, 420)
(2, 114, 2588)
(2, 95, 3014)
(1, 193, 2261)
(3, 248, 1545)
(3, 197, 1829)
(3, 310, 900)
(2, 257, 1728)
(0, 1, 3

## C++

The blog post also came with a C++ implementation. Only a small bug was fixed, so that C++ and Python version have identical simulations.

C++ needs be compiled first

In [8]:
!g++ example.cpp -o example -std=c++11 -O3

zsh:1: command not found: g++


Then we can use unix helper `perf` to measure performance of the compiled binary.

In [9]:
!perf stat -r 50 -ddd ./example $TIMESTEPS

ok 15, 234, 220
ok 5, 361, 446
ok 5, 361, 446
ok 5, 361, 446
ok 5, 361, 446
ok 5, 361, 446
ok 5, 361, 446
ok 5, 361, 446
ok 5, 361, 446
ok 5, 361, 446
ok 5, 361, 446
ok 5, 361, 446
ok 5, 361, 446
ok 5, 361, 446
ok 5, 361, 446
ok 6, 293, 998
ok 6, 293, 998
ok 6, 293, 998
ok 6, 293, 998
ok 6, 293, 998
ok 6, 293, 998
ok 6, 293, 998
ok 6, 293, 998
ok 6, 293, 998
ok 6, 293, 998
ok 6, 293, 998
ok 6, 293, 998
ok 6, 293, 998
ok 6, 293, 998
ok 6, 293, 998
ok 9, 263, 566
ok 9, 263, 566
ok 9, 263, 566
ok 9, 263, 566
ok 9, 263, 566
ok 9, 263, 566
ok 9, 263, 566
ok 9, 263, 566
ok 9, 263, 566
ok 9, 263, 566
ok 9, 263, 566
ok 9, 263, 566
ok 9, 263, 566
ok 9, 263, 566
ok 9, 263, 566
ok 9, 263, 566
ok 5, 272, 1039
ok 5, 272, 1039
ok 5, 272, 1039
ok 5, 272, 1039

 Performance counter stats for './example 2000' (50 runs):

             70,02 msec task-clock:u                     #    1,067 CPUs utilized            ( +-  1,04% )
                 0      context-switches:u               #    0,000 /sec     

## Julia

Julia is not Object Oriented like Python, hence we need to restructure the class hierarchy.

Luckily it is super simple, as Predator, Prey and Plant don't really make use of Object Orientation inheritance anyway.

In [13]:
%%julia
# Base.@kwdef is a nice little helper which enables default arguments and construction by keywords
Base.@kwdef mutable struct Agent
    vmax::Float64 = 2.5
    world_width::Int = 10
    world_height::Int = 10
    x::Int = rand(0:world_width)
    y::Int = rand(0:world_height)
    
    # initial velocity
    dx::Float64 = 0.0
    dy::Float64 = 0.0

    # inital values
    is_alive::Bool = true
    target::Union{Nothing, Agent} = nothing
    age::Int = 0
    energy::Int = 0
end

Predator(; kwargs...) = Agent(; vmax = 2.5, kwargs...)
Prey(; kwargs...) = Agent(; vmax = 2.0, kwargs...)
Plant(; kwargs...) = Agent(; vmax = 0.0, kwargs...)

Plant (generic function with 1 method)

In [24]:
%%julia
function update!(self, food)
    self.age = self.age + 1

    # we can't move
    if self.vmax == 0.0
        return
    end

    # target is dead, don't chase it further
    if self.target !== nothing && !self.target.is_alive
        self.target = nothing
    end
    
    # eat the target if close enough
    if self.target !== nothing
        squared_dist = (self.x - self.target.x) ^ 2 + (self.y - self.target.y) ^ 2
        if squared_dist < 400
            self.target.is_alive = false
            self.energy = self.energy + 1
        end
    # agent doesn't have a target, find a new one
    else
        min_dist = 9999999
        min_agent = nothing
        for a in food
            if a !== self && a.is_alive
                sq_dist = (self.x - a.x) ^ 2 + (self.y - a.y) ^ 2
                if sq_dist < min_dist
                    min_dist = sq_dist
                    min_agent = a
                end
            end
        end
        if min_dist < 100000
            self.target = min_agent
        end
    end

    # initalize 'forces' to zero
    fx = 0.0
    fy = 0.0

    # move in the direction of the target, if any
    if self.target !== nothing
        fx += 0.1 * (self.target.x - self.x)
        fy += 0.1 * (self.target.y - self.y)
    end

    # update our direction based on the 'force'
    self.dx = self.dx + 0.05 * fx
    self.dy = self.dy + 0.05 * fy

    # slow down agent if it moves faster than it max velocity
    velocity = sqrt(self.dx ^ 2 + self.dy ^ 2)
    if velocity > self.vmax
        self.dx = (self.dx / velocity) * (self.vmax)
        self.dy = (self.dy / velocity) * (self.vmax)
    end

    # update position based on delta x/y
    self.x = self.x + Int(round(self.dx))
    self.y = self.y + Int(round(self.dy))

    # ensure it stays within the world boundaries
    self.x = max(self.x, 0)
    self.x = min(self.x, self.world_width)
    self.y = max(self.y, 0)
    self.y = min(self.y, self.world_height)
end

update! (generic function with 2 methods)

In [15]:
%%julia
function main()
    kwargs = (
        world_width = WORLD_WIDTH,
        world_height = WORLD_HEIGHT,
    )
    preys = [Prey(; kwargs...) for i in 1:10]
    predators = [Predator(; kwargs...) for i in 1:10]
    plants = [Plant(; kwargs...) for i in 1:100]

    timestep = 0
    while timestep < TIMESTEPS
        # update all agents
        #[f.update([]) for f in plants]  # no need to update the plants; they do not move
        for a in preys
            update!(a, plants)
        end
        for a in predators
            update!(a, preys)
        end

        # handle eaten and create new plant
        # plants = [p for p in plants if p.is_alive]
        filter!(p -> p.is_alive, plants)
        append!(plants, [Plant(; kwargs...) for i in 1:2])

        # handle eaten and create new preys
        # preys = [p for p in preys if p.is_alive]
        filter!(p -> p.is_alive, preys)
        for p in preys
            if p.energy > 5
                p.energy = 0
                push!(preys, Prey(; x = p.x + rand(-20:20), y = p.y + rand(-20:20), kwargs...))
            end
        end

        # handle old and create new predators
        # predators = [p for p in predators if p.age < 2000]
        filter!(p -> p.age < 2000, predators)
        for p in predators
            if p.energy > 10
                p.energy = 0
                push!(predators, Predator(; x = p.x + rand(-20:20), y = p.y + rand(-20:20), kwargs...))
            end
        end

        # write data to output file
        #[print(timestep, ',', 'Position', ',', 'Predator', ',', a.x, ',', a.y, file=f) for a in predators]
        #[print(timestep, ',', 'Position',  ',', 'Prey', ',', a.x, ',', a.y, file=f) for a in preys]
        #[print(timestep, ',', 'Position',  ',', 'Plant', ',', a.x, ',', a.y, file=f) for a in plants]

        timestep += 1
    end
    println("$(length(predators)), $(length(preys)), $(length(plants))")
end

main (generic function with 1 method)

In [16]:
%julia using BenchmarkTools
%julia @btime main()

3, 292, 1243
2, 335, 958
7, 302, 671
4, 388, 227
3, 274, 1378
4, 360, 411
1, 252, 1675
1, 0, 3934
2, 213, 2059
4, 299, 902
9, 320, 299
3, 409, 249
5, 339, 579
5, 297, 771
0, 194, 2472
6, 316, 685
4, 245, 1267
4, 329, 838
4, 265, 1240
2, 395, 299
5, 369, 305
3, 367, 393
5, 300, 833
2, 169, 2394
0, 55, 3465
0, 1, 4043
1, 348, 918
1, 98, 2768
5, 337, 505
5, 277, 975
0, 140, 2635
2, 224, 1884
2, 344, 693
0, 2, 4020
1, 209, 2130
2, 259, 1710
6, 339, 254
1, 231, 1904
2, 401, 298
1, 57, 3320
8, 311, 572
2, 363, 496
3, 257, 1254
1, 328, 910
3, 308, 1032
3, 384, 305
3, 331, 684
4, 250, 1206
5, 362, 323
1, 290, 1325
1, 319, 908
3, 334, 838
2, 397, 273
1, 162, 2566
3, 328, 842
5, 351, 309
3, 287, 1286
3, 374, 466
2, 378, 279
0, 348, 1231
0, 132, 2911
4, 363, 449
1, 235, 1663
5, 349, 281
4, 382, 249
1, 235, 1778
3, 375, 564
3, 201, 1973
1, 350, 1022
6, 360, 285
3, 265, 1226
5, 364, 337
4, 377, 318
3, 416, 281
0, 0, 4068
5, 313, 822
3, 312, 926
1, 173, 2413
3, 373, 432
8, 288, 413
5, 355, 378
5, 36

In [17]:
%julia @benchmark main()

0, 0, 3961
4, 332, 797
2, 230, 1847
2, 304, 1239
2, 101, 2916
0, 0, 4046
1, 243, 1944
3, 216, 1687
3, 261, 1420
4, 375, 350
4, 257, 1369
6, 298, 470
0, 1, 4007
4, 313, 831
0, 442, 242
6, 348, 221
7, 310, 280
5, 355, 260
1, 290, 1510
5, 391, 265
5, 324, 464
0, 2, 3890
2, 382, 372
7, 326, 358
2, 385, 306
8, 334, 236
8, 305, 533
6, 330, 328
3, 368, 585
3, 338, 767
2, 311, 946
5, 313, 552
4, 297, 985
2, 402, 429
3, 360, 345
1, 258, 1656
6, 332, 321
0, 324, 1328
0, 251, 1873
1, 267, 1412
0, 2, 3971
0, 2, 3988
5, 382, 316
2, 379, 396
0, 244, 1806
1, 418, 440
3, 386, 291
2, 422, 199
2, 213, 1734
3, 237, 1579
1, 1, 3806
4, 370, 510
1, 214, 1996
2, 332, 964
0, 175, 2451
2, 368, 566
1, 241, 1955
2, 288, 1285
2, 381, 278
2, 302, 1212
3, 315, 977
6, 339, 310
4, 357, 433
0, 340, 1179
1, 57, 3393
4, 226, 1696
2, 285, 1370
5, 337, 485
3, 251, 1679
3, 412, 268
0, 1, 3952
4, 300, 1010
4, 165, 2144
3, 392, 241
3, 176, 2012
2, 308, 928
3, 318, 799
1, 223, 1947
4, 327, 692
3, 191, 1931
1, 238, 1978
0, 0, 

BenchmarkTools.Trial: 94 samples with 1 evaluation.
 Range (min … max):   6.911 ms … 73.762 ms  ┊ GC (min … max): 0.00% … 1.03%
 Time  (median):     59.886 ms              ┊ GC (median):    1.30%
 Time  (mean ± σ):   53.598 ms ± 17.712 ms  ┊ GC (mean ± σ):  1.53% ± 0.93%

  ▁                                             ▇ ▃▂█          
  █▃▁▃▁▁▃▁▁▁▁▁▁▁▁▄▁▃▁▃▁▁▁▃▁▁▃▃▃▁▃▁▁▁▄▁▁▁▁▃▁▃▃▆█▇█▇███▇▇▃▃▇▃▁▃ ▁
  6.91 ms         Histogram: frequency by time        73.1 ms <

 Memory estimate: 1.30 MiB, allocs estimate: 11775.

# Thank you for participating ❤️

You can always reach me at stephan.sahm@jolin.io or <a style="justify-content: center; padding: 7px; text-align: center; outline: none; text-decoration: none !important; color: #ffffff !important; width: 200px; height: 32px;border-radius: 16px; background-color: #0A66C2;" href="https://www.linkedin.com/company/jolin-io/" target="_blank">Follow on LinkedIn</a>
<!-- alternative follow https://www.linkedin.com/comm/mynetwork/discovery-see-all?usecase=PEOPLE_FOLLOWS&followMember=stephan-sahm-918656b7 -->

<a href="https://www.jolin.io" target="_blank" rel="noreferrer noopener">
<img src="https://www.jolin.io/assets/Jolin/Jolin-Banner-Website-v1.3-darkmode.webp">
</a>